# Альбументации

In [ ]:
%load_ext autoreload
%autoreload 2

import os 

PROJECT_DPATH = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_DPATH = os.path.join(PROJECT_DPATH, "data")

import torch
import torchvision
from torchvision.transforms import transforms
import numpy as np
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import albumentations as albu
import cv2

from mnist_recognition.transforms import Invertor, Convertor, AlbuAugmentation

## Загрузка тестовой выборки

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
   DATA_DPATH, train=False, transform=Invertor(), download=True
)

# Создаём лоядеры данных.
# так как модель ожидает данные в определённой форме
test_dataloader=torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=1,
    shuffle=False
)

In [ ]:
images, labels = [], []
for info in test_data:
    img, label = info 
    images.append(np.array(img))
    labels.append(label)

Альбументация данных - это такой подход, который используется для расширения обучающей выборки за счёт небольших изменений исходных данных. Например, сдвига данных, отражения по вертикальной/горизонтальной оси, изменение насыщенности пикселей (в случае картинок). 

Рассмотрим виды альбументаций, которые будем применять к данным 

## Примеры альбументаций

### 1) Альбументация - Смещение 

In [ ]:
transform_shift = albu.ShiftScaleRotate(
    shift_limit=0.2,
    scale_limit=0,
    rotate_limit=0,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_shift = transform_shift(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_shift, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

### 2) Альбументация - Изменение масштаба 

In [ ]:
transform_scale = albu.ShiftScaleRotate(
    shift_limit=0,
    scale_limit=0.25,
    rotate_limit=0,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_scale = transform_scale(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_scale, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

### 3) Поворот изображения на небольшой угол (возьмем границы угла поворота -20 до 20)

In [ ]:
transform_rotate = albu.ShiftScaleRotate(
    shift_limit=0,
    scale_limit=0,
    rotate_limit=20,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_rotate = transform_rotate(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_rotate, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

### 4) Применение 3х альбументаций одновременно: Поворот, изменение масштаба, смещение

In [ ]:
transform_all = albu.ShiftScaleRotate(
    shift_limit=0.15,
    scale_limit=0.2,
    rotate_limit=20,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_all = transform_all(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_all, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

### 5) Размытие изображения

In [ ]:
transform_blur = albu.GaussianBlur(blur_limit=0.1, p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_blure = transform_blur(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_blure, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

### 6) Изменение яркости и контрастности 

In [ ]:
transform_bright =  albu.RandomBrightnessContrast (
    brightness_limit=0.8, 
    contrast_limit=0.25, 
    brightness_by_max=True, 
    always_apply=False, 
    p=0.9
)


@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img_bright = transform_bright(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img_bright, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

## Проверка альбументаций на обучающей выборке

In [ ]:
data_transform = transforms.Compose(
    [Invertor(), Convertor(), AlbuAugmentation(), transforms.ToTensor()]
)
    # загружаем обучающую выборку
train_data = torchvision.datasets.MNIST(
    DATA_DPATH,
    train=True,
    transform=data_transform,
    download=True,
)


Посмотрим, что альбументации применились в данным  

In [ ]:
@interact
def show_images(index=IntSlider(val=0, min=0, max=len(train_data)-1)):
    img,_ = train_data[index]
    img = np.array(img).reshape(28,28)

    plt.figure(figsize=[4, 4])
    plt.imshow(img, cmap="gray")

    plt.show()